In [1]:
import pandas as pd
import numpy as np

In [2]:
dataInc = pd.read_excel(r'C:\Users\MR38804\Documents\Network Graph\Data\Top Entities List + Dataset Final2.xlsx', sheet_name='Trx Incoming')

In [3]:
dataOut = pd.read_excel(r'C:\Users\MR38804\Documents\Network Graph\Data\Top Entities List + Dataset Final2.xlsx', sheet_name='Trx Outgoing')

In [ ]:
dataInc

In [5]:
dataInc.drop(['BASE DATE', 'COUNTRY (TT)'], inplace=True, axis=1)
dataInc.columns = ['DEBTOR GCIF', 'DEBTOR CIF', 'AMOUNT_TX_IDR', 'SENDER/RECIPIENT ACCOUNT', 'SENDER/RECIPIENT NAME', 'SENDER/RECIPIENT BANK', 'DEBTOR ACCOUNT', 'DEBTOR NAME', 'DEBTOR BANK', 'LOB', 'CHANNEL', 'TYPE']

In [ ]:
dataOut

In [7]:
dataOut.drop(['BASE DATE', 'COUNTRY (TT)'], inplace=True, axis=1)
dataOut.columns = ['DEBTOR GCIF', 'DEBTOR CIF', 'AMOUNT_TX_IDR', 'DEBTOR ACCOUNT', 'DEBTOR NAME', 'DEBTOR BANK', 'SENDER/RECIPIENT ACCOUNT', 'SENDER/RECIPIENT NAME', 'SENDER/RECIPIENT BANK', 'LOB', 'CHANNEL', 'TYPE']

In [8]:
data = pd.concat([dataOut, dataInc])

In [ ]:
data

In [10]:
data['DEBTOR BANK'].value_counts()

BANK BII MAYBANK    24980
Name: DEBTOR BANK, dtype: int64

In [ ]:
data.loc[data['SENDER/RECIPIENT NAME'].str.contains('AVOLON')]

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24980 entries, 0 to 17070
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DEBTOR GCIF               24977 non-null  object 
 1   DEBTOR CIF                24449 non-null  float64
 2   AMOUNT_TX_IDR             24980 non-null  float64
 3   DEBTOR ACCOUNT            24936 non-null  float64
 4   DEBTOR NAME               24977 non-null  object 
 5   DEBTOR BANK               24980 non-null  object 
 6   SENDER/RECIPIENT ACCOUNT  8165 non-null   float64
 7   SENDER/RECIPIENT NAME     24980 non-null  object 
 8   SENDER/RECIPIENT BANK     24980 non-null  object 
 9   LOB                       24980 non-null  object 
 10  CHANNEL                   24980 non-null  object 
 11  TYPE                      24980 non-null  object 
dtypes: float64(4), object(8)
memory usage: 2.5+ MB


In [ ]:
data.fillna('UNKNOWN')

In [14]:
gpd_data = data.groupby(['DEBTOR GCIF', 'DEBTOR NAME', 'DEBTOR BANK', 'SENDER/RECIPIENT NAME', 'SENDER/RECIPIENT BANK', 'TYPE'])['AMOUNT_TX_IDR'].agg(AMOUNT_TX_IDR = 'sum',TRX='count')

In [15]:
gpd_data.reset_index(inplace=True)

In [ ]:
gpd_data

In [ ]:
gpd_data.loc[gpd_data['SENDER/RECIPIENT NAME'].str.contains('AVOLON')]

In [18]:
gpd_data.to_excel('GPD_Data_v1.xlsx', index=False)

In [19]:
current_data = gpd_data

In [20]:
list_entity=current_data['DEBTOR NAME'].values.tolist()
list_entity+=current_data['SENDER/RECIPIENT NAME'].values.tolist()
list_entity=pd.DataFrame({
    'ENTITY NAME':list_entity
})
list_entity=list_entity.drop_duplicates(keep='first')

In [ ]:
list_entity

In [ ]:
list_entity.loc[list_entity['ENTITY NAME'].str.contains('AVOLON')]

In [ ]:
entity_DEBTOR=list(set(current_data['DEBTOR NAME']))
all_entity=list_entity['ENTITY NAME'].values.tolist()
sr_entity_data=[]
for name in all_entity:
    placeholder=''
    as_DEBTOR=current_data.loc[current_data['DEBTOR NAME']==name]
    as_other  =current_data.loc[current_data['SENDER/RECIPIENT NAME']==name]
    
    #if in DEBTOR, incoming -> receiver with Maybank bank, outgoing -> sender with Maybank bank
    if len(as_DEBTOR)!=0: #pernah menjadi customer maybank
        if 'INCOMING' in as_DEBTOR['TYPE'].values and 'OUTGOING' in as_DEBTOR['TYPE'].values:
            sr_entity_data.append([name,'Both',as_DEBTOR['DEBTOR BANK'].iloc[0]])
        elif 'INCOMING' in as_DEBTOR['TYPE'].values:
            sr_entity_data.append([name,'Receiver',as_DEBTOR['DEBTOR BANK'].iloc[0]])
        elif 'OUTGOING' in as_DEBTOR['TYPE'].values:
            sr_entity_data.append([name,'Sender',as_DEBTOR['DEBTOR BANK'].iloc[0]])
    
    #if in sender/recipient, incoming -> sender with SR bank name, outgoing -> recipient with SR bank name
    if len(as_other)!=0: #pernah transaksi dgn rek Maybank sebagai nasabah bank lain
        sr_bank_names=list(set(as_other['SENDER/RECIPIENT BANK']))
        for bank in sr_bank_names:
            as_other_bank=as_other.loc[as_other['SENDER/RECIPIENT BANK']==bank]
            if 'INCOMING' in as_other_bank['TYPE'].values and 'OUTGOING' in as_other_bank['TYPE'].values:
                sr_entity_data.append([name,'Both',bank])
            elif 'INCOMING' in as_other_bank['TYPE'].values:
                sr_entity_data.append([name,'Sender',bank])
            elif 'OUTGOING' in as_other_bank['TYPE'].values:
                sr_entity_data.append([name,'Receiver',bank])

sr_entity_df=pd.DataFrame(sr_entity_data,columns=['NAME','DIRECTION','BANK'])
sr_entity_df

In [24]:
sr_entity_df.loc[sr_entity_df['NAME'].str.contains('AVOLON')]

,NAME,DIRECTION,BANK


In [ ]:
#merge gcif
gcif_copying=current_data[['DEBTOR NAME','DEBTOR GCIF']]
gcif_copying.columns=['NAME','GCIF']

sr_entity_df=sr_entity_df.merge(gcif_copying,on='NAME',how='left')
sr_entity_df=sr_entity_df.fillna('External')

sr_entity_df=sr_entity_df.drop_duplicates()
sr_entity_df

### Create entity data - Coloring

- Internal
 - In: orange
 - Out: purple
 - Both: lightblue
- External
 - In: black 
 - Out: red
 - Both: lightgreen

In [26]:
row=[sr_entity_df.iloc[0]['NAME'],sr_entity_df.iloc[0]['DIRECTION'],sr_entity_df.iloc[0]['BANK']]
'MAYBANK' in row[2]

True

In [ ]:
#'MBI Out','MBI In','MBI Both','Non-MBI Out','Non-MBI In','Non-MBI Both'),
# color=c("#3333ff","#00bf30","orange","#685b5b","#c071f1","#ff0000"))
final_entity_df=[]
for i in range(len(sr_entity_df)):
    row=sr_entity_df.iloc[i]
    row=[row['NAME'],row['DIRECTION'],row['BANK'],row['GCIF']
         #,row['LOB']
        ]
    
    #row color (based on internal/external)
    if 'MAYBANK' in row[2]: #internal
        if row[1]=='Both':
            row+=['MBI Both','orange']
        elif row[1]=='Receiver':
            row+=['MBI In','#00bf30']
        else:
            row+=['MBI Out','#3333ff']
    else:
        if row[1]=='Both':
            row+=['Non-MBI Both','#ff0000']
        elif row[1]=='Receiver':
            row+=['Non-MBI In','#c071f1']
        else:
            row+=['Non-MBI Out','#685b5b']
    
    #row shape (semuanya dot) -> uncomment if needed
    row+=['dot']
    #row shape (tergantung lob) -> uncomment if needed
    #if row[4] == 'Consumer':
    #    row+=['square']
    #elif row[4] == 'RSME':
    #    row+=['triangle']
    #elif row[4] == 'SME+':
    #    row+=['triangleDown']
    #elif row[4] == 'Corporate':
    #    row+=['star']
    #elif row[4] == 'Commercial (BB)':
    #    row+=['diamond']
    #else :
    #    row+=['dot']

    
    #row id (tambahin i)
    row+=[i]
    
    #append
    final_entity_df.append(row)
    
final_entity_df=pd.DataFrame(final_entity_df,
                             columns=['ENTITY','DIRECTION','SOURCE','GCIF',
                                      #'LOB',
                                      'TYPE','COLOR','SHAPE','ID'])
final_entity_df=final_entity_df.drop_duplicates()
final_entity_df

In [28]:
final_entity_df.loc[final_entity_df['ENTITY'].str.contains('AVOLON')]

,ENTITY,DIRECTION,SOURCE,GCIF,TYPE,COLOR,SHAPE,ID


In [ ]:
trans_id=[]
for i in range(len(current_data)):
    row=current_data.iloc[i]
    
    if row['TYPE']=='INCOMING': #DEBTOR nerima, sr ngirim
        trans_id.append([row['SENDER/RECIPIENT NAME'],row['SENDER/RECIPIENT BANK'],
                         row['DEBTOR NAME'],row['DEBTOR BANK'],
                         row['AMOUNT_TX_IDR'],row['TRX']])
    else: #DEBTOR ngirim, sr nerima
        trans_id.append([row['DEBTOR NAME'],row['DEBTOR BANK'],
                         row['SENDER/RECIPIENT NAME'],row['SENDER/RECIPIENT BANK'],
                         row['AMOUNT_TX_IDR'],row['TRX']])
    
#create trans_df
trans_id=pd.DataFrame(trans_id,columns=['FROM_ENTITY','FROM_SOURCE','TO_ENTITY','TO_SOURCE','AMOUNT_TX_IDR','TRX'])

#get FROM_ID using entity data
temp=final_entity_df[['ENTITY','SOURCE','ID']].copy()
temp.columns=['FROM_ENTITY','FROM_SOURCE','FROM_ID']
trans_id=trans_id.merge(temp,on=['FROM_ENTITY','FROM_SOURCE'],how='left')

#get TO_ID using entity data
temp=final_entity_df[['ENTITY','SOURCE','ID']].copy()
temp.columns=['TO_ENTITY','TO_SOURCE','TO_ID']
trans_id=trans_id.merge(temp,on=['TO_ENTITY','TO_SOURCE'],how='left')

#trans_id=trans_id[['FROM_ID','TO_ID','AMOUNT_TX_IDR']]
trans_id=trans_id.drop_duplicates()
trans_id

In [30]:
#create initial size
size_amount=[]
size_count=[]
size_chain=[]
for name in final_entity_df['ID'].values:
    curr_agg=trans_id.loc[(trans_id['FROM_ID']==name)|(trans_id['TO_ID']==name)]
    size_amount.append(sum(curr_agg['AMOUNT_TX_IDR']))
    size_count.append(sum(curr_agg['TRX']))
    size_chain.append(len(curr_agg))

final_entity_df['ORIG_AMOUNT']=size_amount
final_entity_df['ORIG_COUNT']=size_count
final_entity_df['ORIG_CHAIN']=size_chain

final_entity_df['SIZE_AMOUNT']=[1 if i<=1 else i for i in size_amount]
final_entity_df['SIZE_COUNT']=[1 if i<=1 else i for i in size_count]
final_entity_df['SIZE_CHAIN']=[1 if i<=1 else i for i in size_chain]

#create individual flags
final_entity_df['FLAG_AMT']=final_entity_df['SIZE_AMOUNT'].apply(lambda x: 'high amt' if x>=34 else 'low amt')
final_entity_df['FLAG_TRX']=final_entity_df['SIZE_COUNT'].apply(lambda x: 'high freq' if x>=4 else 'low freq')
final_entity_df['FLAG_CHAIN']=final_entity_df['SIZE_CHAIN'].apply(lambda x: 'high chain' if x>=10 else 'low chain')

#create final filter
final_filter=[]
simplified=[]
for i in range(len(final_entity_df)):
    row=final_entity_df.iloc[i]
    curr_filter=''
    
    #final filter
    if row['FLAG_AMT']=='high amt':
        curr_filter+='high amt, '
    if row['FLAG_TRX']=='high freq':
        curr_filter+='high freq, '
    if row['FLAG_CHAIN']=='high chain':
        curr_filter+='high chain, '
    if curr_filter=='':
        curr_filter='low, '
    
    #simplified view
    if row['SIZE_CHAIN']>=2:
        curr_filter+='simplified'
        simplified.append('simplified')
    else:
        simplified.append('full')
        
    #remove trailing comma and whitespace
    curr_filter=curr_filter.rstrip(' ')
    curr_filter=curr_filter.rstrip(',')
    
    #append
    final_filter.append(curr_filter+'')

final_entity_df['FILTER']=final_filter
final_entity_df['SIMPLIFIED']=simplified
#final_entity_df['FLAG_LOB']=final_entity_df['LOB']+', '+final_entity_df['SIMPLIFIED']

In [ ]:
final_entity_df

In [ ]:
final_entity_df.loc[final_entity_df['ENTITY'].str.contains('GATEWAY CONTAINER LINE')]

In [33]:
final_entity_df.loc[final_entity_df['ENTITY'].str.contains('AVOLON')]

,ENTITY,DIRECTION,SOURCE,GCIF,TYPE,COLOR,SHAPE,ID,ORIG_AMOUNT,ORIG_COUNT,ORIG_CHAIN,SIZE_AMOUNT,SIZE_COUNT,SIZE_CHAIN,FLAG_AMT,FLAG_TRX,FLAG_CHAIN,FILTER,SIMPLIFIED


In [34]:
final_entity_df.to_csv('entity_idv3.csv')
trans_id.to_csv('trans_idv3.csv')